In [ ]:
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta
from lsst.daf.butler import Butler
import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData

In [ ]:
butler = Butler('/repo/embargo', collections=['LSSTCam/raw/all', 
                                            'LSSTCam/calib/unbounded', 'LSSTCam/runs/nightlyValidation', 
                                             'LSSTCam/runs/nightlyValidation/20250425/w_2025_17/DM-50157'])
instrument = 'LSSTCam'
client = makeEfdClient()

In [ ]:
startDay = 20250527
endDay = 20250608
dayObs = startDay
enableDict = {}
while dayObs <= endDay:
    start = Time(f"{dayObsIntToString(dayObs)}T12:00:00")
    end = Time(f"{dayObsIntToString(calcNextDay(dayObs))}T12:00:00")
    states = getEfdData(
        client,
        "lsst.sal.MTMount.logevent_summaryState",
        columns=['summaryState'],
        begin=start,
        end=end
    )
    enables = states[states['summaryState'] == 2]
    print(f"There were {len(enables)} enable events on {dayObs}")
    enableDict[dayObs] = enables
    dayObs = calcNextDay(dayObs)

In [ ]:
filename = "/home/c/cslage/u/LSSTCam/data/pointing_results_rot_dayObs_27may25-08jun25.pkl"
with open(filename, 'rb') as f:
    [els, azs, ras, decs, rots, deltaRas, deltaDecs, pressures, temps, hums, times, filters, dayObss] = pkl.load(f)
len(els)

In [ ]:
outfilename = "/home/c/cslage/u/LSSTCam/data/Test_14Jun25.dat"
outfile = open(outfilename, 'w')
outfile.write("!" + outfilename + "\n")
outfile.write("!Simonyi Telescope file,June 14, 2025 \n")
outfile.write(": ALTAZ\n")
outfile.write(": ROTNR\n")
outfile.write("-30 14 40.2\n")
lastDayObs = 0
for i in range(len(els)):
    dayObs = dayObss[i]
    enables = enableDict[dayObs]
    thisTime = Time(times[i], format='mjd')
    if dayObs != lastDayObs:
        lastDayObs = dayObs
        initialStateChanges = 0
        for j in range(len(enables)):
            stateChangeTime = Time(enables.index[j])
            if thisTime > stateChangeTime:
                print("Initial", initialStateChanges, stateChangeTime.isot)
                initialStateChanges += 1
    if azs[i] > 360.0:
        azs[i] -= 360.0
    if azs[i] < 0.0:
        azs[i] += 360.0
    stateChanges = -initialStateChanges
    for j in range(len(enables)):
        stateChangeTime = Time(enables.index[j])
        if thisTime > stateChangeTime:
            stateChanges += 1
            print("Final", stateChanges, stateChangeTime.isot, thisTime.isot)
    outfile.write(f"{azs[i]:.9f}\t{els[i]:.9f}\t{dayObs}\t{thisTime.isot}  {stateChanges}\n")
outfile.write("END\n")
outfile.close()



In [ ]:
len(enables)